In [73]:
import requests
import signal  
import sys  
import asyncio  
import aiohttp  
import json
from lxml import html  

import numpy as np
from couchbase.cluster import Cluster
from couchbase.cluster import PasswordAuthenticator
from couchbase import FMT_BYTES
from lxml import html  
import requests
from time import sleep
import json
import argparse
from collections import OrderedDict
from time import sleep
import csv

cluster = Cluster('couchbase://localhost')
authenticator = PasswordAuthenticator('Frank', 'frank123')
cluster.authenticate(authenticator)
bucket = cluster.open_bucket('stocks')

In [79]:
def fetch(session, ticker):
    base_url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
    with session.get(base_url) as response:
        if response.status_code != 200:
            print("FAILURE::{0}".format(base_url))
        else:
            data = response.text
            parser = html.fromstring(data)
            summary_data = OrderedDict()
            try:
                quote = parser.xpath('//span[contains(@class,"Trsdu(0.3s) Fw(b)")]')[0].text
                quote_currency = parser.xpath('//div[contains(@class,"C($tertiaryColor) Fz(12px)")]//span')[0].text.split()[-1]
            except:
                print ("Failed to parse quote")
                return {"error":"Failed to parse quote"}
            
    summary_url = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
    summary_data = OrderedDict()
    with session.get(summary_url) as response:
        if response.status_code != 200:
            print("FAILURE:{0}".format(url))
        else:
            data = response.text
            json_loaded_summary = json.loads(data)
            try:
                quoteSummary = json_loaded_summary["quoteSummary"]["result"][0]
                financial_currency=quoteSummary["financialData"]["financialCurrency"]
                eps = quoteSummary["defaultKeyStatistics"]["trailingEps"]['raw']
                book_value = quoteSummary["defaultKeyStatistics"]["bookValue"]['raw']
            except:
                print ("Failed to parse json response")
                return {"error":"Failed to parse json response"}
            
    summary_data.update({'quote':quote,'eps':eps,'book_value':book_value,
            'quote_currency':quote_currency,'financial_currency':financial_currency})
    
    bucket.upsert(ticker,summary_data)


In [83]:
from datetime import datetime

def get_data_synchronous():
    tickers_to_fetch = [
        "RE4.SI",' fdfd',
        "D05.SI",
        "5WH.SI",
        "UD2.SI",
    ]

    with requests.Session() as session:
        print("{0:<30} {1:>20}".format("Ticker", "Completed at"))
        
        total_start_time = datetime.now()
        for ticker in tickers_to_fetch:
            fetch(session, ticker)
            elapsed = datetime.now() - total_start_time
            time_completed_at = "{}s".format(elapsed)
            print("{0:<30} {1:>20}".format(ticker, time_completed_at))

In [84]:
get_data_synchronous()

Ticker                                 Completed at
RE4.SI                              0:00:00.789509s
Failed to parse quote
 fdfd                               0:00:00.992364s
D05.SI                              0:00:02.037940s
5WH.SI                              0:00:02.563894s
UD2.SI                              0:00:03.504932s


In [95]:
import requests
import asyncio
from concurrent.futures import ThreadPoolExecutor

START_TIME = datetime.now()

def fetch(session, ticker):
    base_url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
    with session.get(base_url) as response:
        if response.status_code != 200:
            print("FAILURE::{0}".format(base_url))
        else:
            data = response.text
            parser = html.fromstring(data)
            summary_data = OrderedDict()
            try:
                quote = parser.xpath('//span[contains(@class,"Trsdu(0.3s) Fw(b)")]')[0].text
                quote_currency = parser.xpath('//div[contains(@class,"C($tertiaryColor) Fz(12px)")]//span')[0].text.split()[-1]
            except:
                print ("Failed to parse quote")
                return {"error":"Failed to parse quote"}
            
    summary_url = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
    summary_data = OrderedDict()
    with session.get(summary_url) as response:
        if response.status_code != 200:
            print("FAILURE::{0}".format(summary_url))
        else:
            data = response.text
            json_loaded_summary = json.loads(data)
            try:
                quoteSummary = json_loaded_summary["quoteSummary"]["result"][0]
                financial_currency=quoteSummary["financialData"]["financialCurrency"]
                eps = quoteSummary["defaultKeyStatistics"]["trailingEps"]['raw']
                book_value = quoteSummary["defaultKeyStatistics"]["bookValue"]['raw']
                summary_data.update({'ticker':ticker,'quote':quote,'eps':eps,'book_value':book_value,
                                     'quote_currency':quote_currency,'financial_currency':financial_currency})

                bucket.upsert(ticker,summary_data)
                elapsed =  datetime.now() - START_TIME
                time_completed_at = "{}s".format(elapsed)
                print("{0:<30} {1:>20}".format(ticker, time_completed_at))
            except:
                print ("Failed to parse json response")
                return {"error":"Failed to parse json response"}
            
    

async def get_data_asynchronous():
    tickers_to_fetch = [i.strip().split(';')[-3].strip()+'.SI' for i in open("./SESprice.dat").readlines()]
    
    print("{0:<30} {1:>20}".format("Ticker", "Completed at"))
    with ThreadPoolExecutor(max_workers=100) as executor:
        with requests.Session() as session:

            loop = asyncio.get_event_loop()
            START_TIME = datetime.now()
            tasks = [
                loop.run_in_executor(
                    executor,
                    fetch,
                    *(session, ticker) 
                )
                for ticker in tickers_to_fetch
            ]
            for response in await asyncio.gather(*tasks):
                pass

def main():
    loop = asyncio.get_running_loop()
    future = asyncio.ensure_future(get_data_asynchronous())
    #loop.run_until_complete(future)

main()

Ticker                                 Completed at
Failed to parse quote
575.SI                              0:00:01.878682s
502.SI                              0:00:01.926838s
FAILURE::https://query2.finance.yahoo.com/v10/finance/quoteSummary/K3HD.SI?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com
541.SI                              0:00:02.142986s
5EF.SI                              0:00:02.207798s
AYV.SI                              0:00:02.459269s
533.SI                              0:00:02.520636s
5TS.SI                              0:00:02.536879sFailed to parse quote

Failed to parse quote
Failed to parse quote
CLN.SI                              0:00:02.738712s
AWG.SI                              0:00:02.744765s
Failed to parse quote
FAILURE::https://query2.finance.yahoo.com/v10/finance/quoteSummary/BQI.SI?formatted=true&lan

Failed to parse json response
Failed to parse quote
FAILURE::https://query2.finance.yahoo.com/v10/finance/quoteSummary/BR9.SI?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com
Failed to parse json response
Failed to parse json response
1H3.SI                              0:00:06.693713s
MU7.SI                              0:00:06.740549s
P8A.SI                              0:00:06.785115s
Failed to parse quote
Failed to parse quote
C41.SI                              0:00:06.880004s
Failed to parse quote
Failed to parse quote
5FH.SI                              0:00:06.940226s
Failed to parse quote
P9D.SI                              0:00:06.976706s
C76.SI                              0:00:07.011004s
Failed to parse quote
Failed to parse quote
567.SI                              0:00:07.085769s
D05.SI                              0:00:

J91U.SI                             0:00:15.157082s
42Z.SI                              0:00:15.168249s
Failed to parse json response
Failed to parse quote
1C0.SI                              0:00:15.341231sFailed to parse json response

Failed to parse quoteS44.SI                              0:00:15.436123s

Failed to parse quote
FAILURE::https://query2.finance.yahoo.com/v10/finance/quoteSummary/5DN.SI?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com
LIW.SI                              0:00:15.908933s
Failed to parse quote
Failed to parse quote
BQD.SI                              0:00:16.188302s
R14.SI                              0:00:16.215014s
BFT.SI                              0:00:16.236633s
Failed to parse quote
F10.SI                              0:00:16.266182s
Q5T.SI                              0:00:16.365890s
D8DU.SI    

BDF.SI                              0:00:21.415100s
H18.SI                              0:00:21.449919s
42N.SI                              0:00:21.470432s
Q0F.SI                              0:00:21.516029s
H12.SI                              0:00:21.568038s
Failed to parse quote
AIY.SI                              0:00:21.600177s
Failed to parse json responseH13.SI                              0:00:21.663220s

Failed to parse quote
Failed to parse json response5TN.SI                              0:00:21.781810s

5I4.SI                              0:00:21.799546s
FAILURE::https://query2.finance.yahoo.com/v10/finance/quoteSummary/QL2.SI?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.comI49.SI                              0:00:21.820981s

Failed to parse json response
Failed to parse json response
RDR.SI                              0:0

L02.SI                              0:00:27.148178s
Failed to parse quote
Failed to parse quote
546.SI                              0:00:27.215341s
M15.SI                              0:00:27.242991s
FAILURE::https://query2.finance.yahoo.com/v10/finance/quoteSummary/5EN.SI?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com
5UF.SI                              0:00:27.266139s
Failed to parse quote
1H4.SI                              0:00:27.333464s
Failed to parse quote
Failed to parse json response
5LY.SI                              0:00:27.389355s
V3M.SI                              0:00:27.404959s
F86.SI                              0:00:27.414318s
Failed to parse json response
DU4.SI                              0:00:27.433618s
Failed to parse quote
OTX.SI                              0:00:27.503070s
CNE.SI                           

T13.SI                              0:00:31.845459s
LUY.SI                              0:00:31.867874s
AP4.SI                              0:00:31.934379s
5G4.SI                              0:00:31.949921s
NEX.SI                              0:00:31.981044s
UV1.SI                              0:00:32.008763s
5WH.SI                              0:00:32.041495s
E8Z.SI                              0:00:32.074647s
S68.SI                              0:00:32.127343s
BAI.SI                              0:00:32.154929s
Failed to parse json response
Failed to parse quote
566.SI                              0:00:32.224602s
Failed to parse quote
S61.SI                              0:00:32.249931s
OYY.SI                              0:00:32.265604s
Failed to parse quoteS59.SI                              0:00:32.313326s

5PL.SI                              0:00:32.346420s
C6L.SI                              0:00:32.409143s
Failed to parse quote
S58.SI                              0:00:32.468891

KUH.SI                              0:00:36.791193s
561.SI                              0:00:36.821041s
M1Z.SI                              0:00:36.841755s
AWI.SI                              0:00:36.854415s
Failed to parse json response
5OQ.SI                              0:00:36.894377s
5G3.SI                              0:00:36.905919s
500.SI                              0:00:36.986639s
K1Q.SI                              0:00:37.023409s
T15.SI                              0:00:37.069860s
T12.SI                              0:00:37.120723s
AGS.SI                              0:00:37.127519s
T41.SI                              0:00:37.194473s
BQM.SI                              0:00:37.197039s
BTM.SI                              0:00:37.214180s
T14.SI                              0:00:37.268142s
Failed to parse quote
T39.SI                              0:00:37.312545s
Failed to parse quote
A50.SI                              0:00:37.358451s
5UX.SI                              0:00:3

Task exception was never retrieved
future: <Task finished coro=<get_data_asynchronous() done, defined at <ipython-input-95-7df9a6ab1ad7>:49> exception=ConnectionError(MaxRetryError("HTTPSConnectionPool(host='finance.yahoo.com', port=443): Max retries exceeded with url: /quote/RMRB.SI?p=RMRB.SI (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1059a9290>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))"))>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/usr/local/lib/python3.7/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/socket.py", line 752, in getaddrinfo
    f

Failed to parse json response
